# Refreshing Optimization + Introduction Numeric Modelling
**Exercise Session Resource Economics (Spring Term 2025)** \
Raul Hochuli (raul.hochuli@unibas.ch)\
Github Repo (https://github.com/raulhochuliunibas/ResEcon_FS25)

In [2]:
# Packages used in the notebook
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import scipy.optimize as opt

## Getting Python

For those of you, who are new to Python, Python is a language which you can use in verious different "programs" / consoles, which are also called IDEs (integrated development environment). Hence there are a number of ways, how to get python running on your system. 
The easiest would most certainly be GoogleColab (basically no prior installments necessary). A slightly more advanced option would be downloading the [Anaconda Navigator](https://docs.anaconda.com/navigator/) which comes with Python and use a IDE that availble there. 

## Refreshing Optimization

Before starting with  numerical optimizations, we first need to get a understanding of how to handle functions in software. As we (should) know, 2D-functions are ”well-behaved” if they are uniquely defined over both x and y-axis. In short, a function f(x) assigns each continuous x-value a unique corresponding y-value. Simply put, this means the function has no kinks, steps or breaks. Now Python is not specifically built to calculate or visualize functions. WolframAlpha or GeoGebra for example are much more usefull tools fort that (which can also derive equation for example). 

But because economics courses (and this one as well) consider *discrete* steps (e.g. yearly steps as x values), Python's array based setup is enough for our purposes. We now try to understand how to link pen and paper calculations to this versatile software. The main point of using *discrete* functions is illustrated in the code example below. 

**1a)**\
Here we plot a graph for the function $f(x) = 2+ x - 0.05x^2$ for the range of $-5 < x < 25$. See how the stepsize in $x$ changes the garnularity in $y$. 

In [3]:
# choose your stepsize
stepsize = 3

# define function
def f(x): 
    return 2+ x - 0.05*(x**2)

# create x values
x_75 = np.arange(-5, 25, 7.5)
x_step = np.arange(-5, 25, stepsize)
x_025 = np.arange(-5, 25, 0.25)

# create y values
y_75 = f(x_75)
y_step = f(x_step)
y_025 = f(x_025)

# create plot
select_mode = 'lines'  #'lines+markers' 
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_75, y=y_75, mode=select_mode, name='stepsize 7.5'))
fig.add_trace(go.Scatter(x=x_step, y=y_step, mode=select_mode, name=f'stepsize {stepsize}'))
fig.add_trace(go.Scatter(x=x_025, y=y_025, mode=select_mode, name='stepsize 0.25'))
fig.update_layout(title='Stepsize comparison', xaxis_title='x', yaxis_title='f(x)', template='plotly_white')

fig.show()


**1b)**\
For another example, you can also use your own knowledge on function derivations to define the derivatives but let Python do the job of finding minima or maxima in your function. For this, take $f(x) = (x+2)(x-1.5)(x+0.75)$ and build a data frame with columns $x$, $f(x)$, $f'(x)$ and $f''(x)$ using a range of $x$ values between -2 and 2 (choosing your own step size). Find the approximate values for $x$ where the function $f(x)$ has a minimum or maximum. 

In [4]:

x = np.arange(-2, 2, 0.01)

def f(x): 
    return (x+2)*(x-1.5)*(x+0.75)
y = f(x)

def f_foc(x): 
    return 3*x**2 + 2.5*x - 2.625
y_foc = f_foc(x)

def f_soc(x): 
    return 6*x + 2.5
y_soc = f_soc(x)


# find the values for x where f'(x) = 0 and f''(x) = 0
df = pd.DataFrame({'x':x, 'f_base':y, "f_foc":y_foc, "f_soc":y_soc})

# select the two rows where f'(x) switches sign
foc_prev_sign_neg = (df['f_foc'] >=0 ) & (df['f_foc'].shift(-1) <= 0) 
foc_prev_sign_pos = (df['f_foc'] <= 0 ) & (df['f_foc'].shift(-1) >= 0)
print(f'range for x where FOC = 0')
print(df.loc[foc_prev_sign_neg | foc_prev_sign_neg.shift(1) | foc_prev_sign_pos | foc_prev_sign_pos.shift(1)])

# select the two rows where f''(x) switches sign
soc_prev_sign_neg = (df['f_soc'] >=0 ) & (df['f_soc'].shift(-1) <= 0)
soc_prev_sign_pos = (df['f_soc'] <= 0 ) & (df['f_soc'].shift(-1) >= 0)
print(f'\n\nrange for x where SOC = 0, ')
print(df.loc[soc_prev_sign_neg | soc_prev_sign_neg.shift(1) | soc_prev_sign_pos | soc_prev_sign_pos.shift(1)])

# visualize function
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='f(x)'))
fig.add_trace(go.Scatter(x=x, y=y_foc, mode='lines', name="f'(x)"))
fig.add_trace(go.Scatter(x=x, y=y_soc, mode='lines', name="f''(x)"))

fig.update_layout(width=600,height=600, template = 'plotly_white')
fig.show()



range for x where FOC = 0
        x    f_base   f_foc  f_soc
55  -1.45  1.135750  0.0575  -6.20
56  -1.44  1.136016 -0.0042  -6.14
260  0.60 -3.159000 -0.0450   6.10
261  0.61 -3.159144  0.0163   6.16


range for x where SOC = 0, 
        x    f_base   f_foc  f_soc
158 -0.42 -1.001088 -3.1458  -0.02
159 -0.41 -1.032546 -3.1457   0.04


## Introduction Numeric Modelling

### 2 Optimal Depletion Problem
Let's consider a cobalt mine "RareMin". The mine operator wants to maximize the net benefits by extracting $q_t$ units of cobalt from the total resource availability of $R_0$ units in the mine over $T$ years ($T = 0, ...,t,... T$), where $T$ is the last year with a positive extraction. The benefit function is given as $\pi_t$ and discount rate as $r$. Assume that all costs of extraction are already included in $\pi$ and any reserves at the end of the period will have a value of zero. 
\begin{align*}
    \pi_t = ln(1+q_t); \quad  R_t - q_{t} = R_{t+1}; \quad \sum_0^{T} q_t = R_0; \quad \rho^t = \frac{1}{(1+r)^t}
\end{align*}


**2a)**\
 Before we dive into how we can solve this question, first take a moment to analyze the different aspects of the problem. How are the individual components defined and what's their range? Use the table below to define all the optimization problem's components.

| **Description**        | **Variable**   | **Range**                  |
|------------------------|----------------|----------------------------|
| Time Indicator / Set   |                |                            |
| Parameter              |                |                            |
| Decision variable      |                |                            |
| State variable         |                |                            |
|-                       |-               |-                            |
| Objective function     |                |                            |
| Constraints            |                |                            |





| **Description**        | **Variable**   | **Range**                  |
|------------------------|----------------|----------------------------|
| Time Indicator / Set   | $t$                  | $[0, ..., T]$                     |
| Parameter              | $r / \rho$           | (constant, $r > -1$)              |
| Decision variable      | $q_t$                | $0 \leq q_t \leq R_t \leq R_0$    |
| State variable         | $R_t$                | $0 \leq R_t \leq R_0$             |
|                        | $\pi_t$              | (non-neg., $0 \leq q_t$)          |
| -                      | -                    | -                                 |
| Objective function     | $\sum_t^T \pi_t$     | (non-neg., $0 \leq q_t$)          |
| Constraints            | $\sum_t^T q_t = / \leq R_0$ | -                                 |


**2b)**\
Now, build an optimization model using [https://scipy.org/](https://scipy.org/) to find the optimal depletion schedule for $q_t$ over $T = 10$ years, with $r = 5\%$ and $R_0 = 100$. Start by
- First defining the set and parameters, 
- Next, use a function to define state variables
- Define the objective function (as it's own function), the array for the decision variable(s) and the decison variable's bounds. 
- Set the constraints of the model (using functions) 
- Run the optimization model and extract the results

What is the sum of all extracted material $\sum(q_t)$ and what is the net present value of the profit over the entire extraction period $\sum_{t=0}^{T} \pi_t  \rho_t$?

In [5]:
# Sets & Parameters --------------------------------
T = 10      # indicator, how many periods
r = 0.05    # parameter, constant over time
R0 = 100    # state variabel, initial value 

# defining an array of discount factors for the entire set of periods
rho = 1/(1+r)**np.arange(T+1)
# rho2 = np.array([(1/(1+r))**t for t in range(T+1)]), # various ways to define the same array

# State Variables -----------------------------------
def f_R(i_q):
    R = np.zeros(T+1)
    R[0] = R0
    for t in range(1, T+1):
        R[t] = R[t-1] - i_q[t-1]
    return R

# Objective Function -------------------------------
# define the objective function as its own function
def f_obj(i_q):
    obj_value = sum([rho[t]* np.log(1+i_q[t]) for t in range(T+1)])
    return -1* obj_value

# Decision Variables -------------------------------
# define an array of values for the decision variable and its bounds
q = np.full(T+1, 1)
bnds = [(0, R0) for t in range(T+1)]

# Constraints --------------------------------------
# define the constraints of the optimization problem, again as a function itself for every constraint
# hint: the constraints should return a value that must either be zero or not zero, dependent on the constraint definition
def c_total_extr(i_q):
    return R0 - sum([i_q[t] for t in range(T+1)])


# here you now define if 
#   > a constraint function must be equal to zero ('eq') 
#   > or a constraint should be greater or equal to zero ('ineq')
constr = [
    {'type': 'eq', 'fun': c_total_extr},
]

# Run the Optimization -----------------------------
results = opt.minimize(f_obj, q, bounds=bnds, constraints=constr,)

# Export results and derive other variables (from decision variable) -----------------------------
q_opt = results.x
R_opt = f_R(q_opt)

df = pd.DataFrame({
    't': range(T+1),
    'q_opt': q_opt,
    'R_opt': R_opt
})

df['profit'] = df.apply(lambda row: np.log(1+row['q_opt']) , axis=1)
df['NPV_profit'] = df['profit'] * rho
print(df)
print(f'\nsum: q={sum(df["q_opt"]):.2f}, \t profit={sum(df["profit"]):.2f} \t NPV_profit={sum(df["NPV_profit"]):.2f}')



     t      q_opt       R_opt    profit  NPV_profit
0    0  11.704474  100.000000  2.541954    2.541954
1    1  11.113133   88.295526  2.494290    2.375515
2    2  10.546396   77.182393  2.446373    2.218933
3    3  10.003354   66.635997  2.398200    2.071655
4    4   9.483131   56.632643  2.349767    1.933159
5    5   8.984901   47.149512  2.301074    1.802952
6    6   8.507865   38.164611  2.252119    1.680566
7    7   8.051178   29.656746  2.202895    1.565556
8    8   7.614081   21.605568  2.153398    1.457505
9    9   7.195828   13.991487  2.103625    1.356016
10  10   6.795659    6.795659  2.053567    1.260712

sum: q=100.00, 	 profit=25.30 	 NPV_profit=20.26


**2c)**\
Now imagine that in each period, the mine operator can only extract $q_t^{max} = 8$. How can you adjust your model to satisfy this requirement? How did the extraction path change? Are all constraint met?

In [6]:
# Sets & Parameters --------------------------------
T = 10      # indicator, how many periods
r = 0.05    # parameter, constant over time
R0 = 100    # state variabel, initial value 

# defining an array of discount factors for the entire set of periods
rho = 1/(1+r)**np.arange(T+1)

# State Variables -----------------------------------
def f_R(i_q):
    R = np.zeros(T+1)
    R[0] = R0
    for t in range(1, T+1):
        R[t] = R[t-1] - i_q[t-1]
    return R

# Objective Function -------------------------------
def f_obj(i_q):
    obj_value = sum([rho[t]* np.log(1+i_q[t]) for t in range(T+1)])
    return -1* obj_value

# Decision Variables -------------------------------
q = np.full(T+1, 1)
bnds = [(0, 8) for t in range(T+1)]  # <-----------Changed upper bound to 8

# Constraints --------------------------------------
def c_total_extr(i_q):
    return R0 - sum([i_q[t] for t in range(T+1)])



constr = [
    {'type': 'ineq', 'fun': c_total_extr},
]

# Run the Optimization -----------------------------
results = opt.minimize(f_obj, q, bounds=bnds, constraints=constr, )

# Export results and derive other variables (from decision variable) -----------------------------
q_opt = results.x
R_opt = f_R(q_opt)

df = pd.DataFrame({
    't': range(T+1),
    'q_opt': q_opt,
    'R_opt': R_opt
})

df['profit'] = df.apply(lambda row: np.log(1+row['q_opt']) , axis=1)
df['NPV_profit'] = df['profit'] * rho
print(df)
print(f'\nsum: q={sum(df["q_opt"]):.2f}, \t profit={sum(df["profit"]):.2f} \t NPV_profit={sum(df["NPV_profit"]):.2f}')


     t  q_opt  R_opt    profit  NPV_profit
0    0    8.0  100.0  2.197225    2.197225
1    1    8.0   92.0  2.197225    2.092595
2    2    8.0   84.0  2.197225    1.992947
3    3    8.0   76.0  2.197225    1.898045
4    4    8.0   68.0  2.197225    1.807662
5    5    8.0   60.0  2.197225    1.721583
6    6    8.0   52.0  2.197225    1.639603
7    7    8.0   44.0  2.197225    1.561526
8    8    8.0   36.0  2.197225    1.487168
9    9    8.0   28.0  2.197225    1.416351
10  10    8.0   20.0  2.197225    1.348905

sum: q=88.00, 	 profit=24.17 	 NPV_profit=19.16


**2d)**\
The local government asks the mine operator to choose a extraction schedule that after $T=10$ years, there are still 27 units of cobalt left in the mine (with no annual extraction limit from before). How can you adjust your model to the requirements?

In [9]:
# Sets & Parameters --------------------------------
T = 10      # indicator, how many periods
r = 0.05    # parameter, constant over time
R0 = 100    # state variabel, initial value 
RT = 27     # state variabel, remains after T periods

# defining an array of discount factors for the entire set of periods
rho = 1/(1+r)**np.arange(T+1)

# State Variables -----------------------------------
def f_R(i_q):
    R = np.zeros(T+1)
    R[0] = R0
    for t in range(1, T+1):
        R[t] = R[t-1] - i_q[t-1]
    return R

# Objective Function -------------------------------
def f_obj(i_q):
    obj_value = sum([rho[t]* np.log(1+i_q[t]) for t in range(T+1)])
    return -1* obj_value

# Decision Variables -------------------------------
q = np.full(T+1, 1)
bnds = [(0, R0) for t in range(T+1)]

# Constraints --------------------------------------
def c_total_extr(i_q):
    return R0 - sum([i_q[t] for t in range(T+1)]) - RT


constr = [
    {'type': 'eq', 'fun': c_total_extr},
]

# Run the Optimization -----------------------------
results = opt.minimize(f_obj, q, bounds=bnds, constraints=constr, )

# Export results and derive other variables (from decision variable) -----------------------------
q_opt = results.x
R_opt = f_R(q_opt)

df = pd.DataFrame({
    't': range(T+1),
    'q_opt': q_opt,
    'R_opt': R_opt
})

df['profit'] = df.apply(lambda row: np.log(1+row['q_opt']) , axis=1)
df['NPV_profit'] = df['profit'] * rho
print(df)
print(f'\nsum: q={sum(df["q_opt"]):.2f}, \t profit={sum(df["profit"]):.2f} \t NPV_profit={sum(df["NPV_profit"]):.2f}')


     t     q_opt       R_opt    profit  NPV_profit
0    0  8.606685  100.000000  2.262459    2.262459
1    1  8.163519   91.393315  2.215230    2.109743
2    2  7.737821   83.229796  2.167661    1.966132
3    3  7.329041   75.491975  2.119748    1.831118
4    4  6.936616   68.162933  2.071487    1.704218
5    5  6.559985   61.226317  2.022869    1.584971
6    6  6.198604   54.666332  1.973887    1.472945
7    7  5.851971   48.467729  1.924536    1.367732
8    8  5.519559   42.615757  1.874807    1.268943
9    9  5.200846   37.096198  1.824686    1.176209
10  10  4.895352   31.895352  1.774164    1.089183

sum: q=73.00, 	 profit=22.23 	 NPV_profit=17.83


**2e)**\
A colleague of yours has also been working on the same project came up with an identical model, but added an additional constraint, making sure the quantity of extraction $q_t$ does not exceed the total amount of available reserves $R_0$. 
```
def c_max_extr_pperiod(i_q):
    R = f_R(i_q)
    return R - i_q
```

You would assume this constraint is non-binding anyway given your model setup and adding it to the model will not change the extraction schedule for $q_t$. Is that correct?

In [11]:
# Sets & Parameters --------------------------------
T = 10      # indicator, how many periods
r = 0.05    # parameter, constant over time
R0 = 100    # state variabel, initial value 

# defining an array of discount factors for the entire set of periods
rho = 1/(1+r)**np.arange(T+1)

# State Variables -----------------------------------
def f_R(i_q):
    R = np.zeros(T+1)
    R[0] = R0
    for t in range(1, T+1):
        R[t] = R[t-1] - i_q[t-1]
    return R

# Objective Function -------------------------------
def f_obj(i_q):
    obj_value = sum([rho[t]* np.log(1+i_q[t]) for t in range(T+1)])
    return -1* obj_value

# Decision Variables -------------------------------
q = np.full(T+1, 1)
bnds = [(0, R0) for t in range(T+1)]

# Constraints --------------------------------------
def c_total_extr(i_q):
    return R0 - sum([i_q[t] for t in range(T+1)])

def c_max_extr_pperiod(i_q):
    R = f_R(i_q)
    return R - i_q

constr = [
    {'type': 'eq', 'fun': c_total_extr},
    {'type': 'ineq', 'fun': c_max_extr_pperiod}
]

# Run the Optimization -----------------------------
results = opt.minimize(f_obj, q, bounds=bnds, constraints=constr, )

# Export results and derive other variables (from decision variable) -----------------------------
q_opt = results.x
R_opt = f_R(q_opt)

df = pd.DataFrame({
    't': range(T+1),
    'q_opt': q_opt,
    'R_opt': R_opt
})

df['profit'] = df.apply(lambda row: np.log(1+row['q_opt']) , axis=1)
df['NPV_profit'] = df['profit'] * rho
print(df)
print(f'\nsum: q={sum(df["q_opt"]):.2f}, \t profit={sum(df["profit"]):.2f} \t NPV_profit={sum(df["NPV_profit"]):.2f}')


     t     q_opt       R_opt    profit  NPV_profit
0    0  9.981119  100.000000  2.396177    2.396177
1    1  9.779258   90.018881  2.377624    2.264404
2    2  9.585951   80.239624  2.359528    2.140161
3    3  9.400888   70.653673  2.341891    2.023014
4    4  9.223751   61.252785  2.324714    1.912548
5    5  9.054234   52.029034  2.307994    1.808374
6    6  8.892032   42.974800  2.291730    1.710124
7    7  8.736869   34.082768  2.275920    1.617454
8    8  8.588468   25.345898  2.260561    1.530037
9    9  8.446552   16.757430  2.245650    1.447566
10  10  8.310878    8.310878  2.231183    1.369753

sum: q=100.00, 	 profit=25.41 	 NPV_profit=20.22


### 3 Bond Example from Refreshing Optimization 

In the first part of the Exercise Refreshing Optimization, I introduced a example of a familiar bond calculation (at least for business and econ students). Remember this is the structure of the bond: nominal value $n$, coupon $c%$, interest rate $r$ and a maturity $T$. The present value is given as 
$$
        PV = \sum_{t=0}^{T-1} \frac{c}{(1+r)^t} + \frac{c+N}{(1+r)^T} 
$$


**3a)**\
The bond with features $n = 20000, c = 3.25\%, T = 10$ is priced at $PV = 18305.1$. Calculate the interest rate / yield for this bond. 

In [20]:
# parameters 
n = 20000
c = 0.0325
T = 10
PV = 18305.1


# define the function evaluating the present value of the bond, dependent on the interest rate / yield
def pv_bond(y):
    pv_calc = sum([c*n/(1+y)**t for t in range(1,T+1)]) + ((c*n) + n )/(1+y)**T
    return pv_calc

# define objective function
def f_obj(y):
    return abs(pv_bond(y) - PV)

# set decision variable and bounds
y_start = 0
bnds = [(0, 1)]

# run optimization
results = opt.minimize(f_obj, y_start, bounds=bnds)

y_opt = results.x[0]

# compute DF and disc CF
cf_val = np.array([c*n for t in range(1,T+1)] + [c*n + n])
npv_val = np.array([cf_val[t]/(1+y_opt)**t for t in range(T+1)])

df = pd.DataFrame({
    't': range(T+1),
    'cf': cf_val,
    'disc_cf': npv_val
})
print(f'Optimal interest rate / yield: {round(y_opt*100,5)}%')
print(df)
print(f'\nsum: cf={sum(df["cf"]):.2f}, \t disc_cf={sum(df["disc_cf"]):.2f}')

Optimal interest rate / yield: 4.58908%
     t       cf       disc_cf
0    0    650.0    650.000000
1    1    650.0    621.479797
2    2    650.0    594.210981
3    3    650.0    568.138646
4    4    650.0    543.210292
5    5    650.0    519.375726
6    6    650.0    496.586955
7    7    650.0    474.798092
8    8    650.0    453.965264
9    9    650.0    434.046523
10  10  20650.0  13184.286720

sum: cf=27150.00, 	 disc_cf=18540.10


### 4) Optimal Depletion Problem (using Pyomo)

So far, we have only used *scipiy.optimize* to run our small optimization models. However there are a large number of other packages and methods out there (also in Python) that you can use. Just to show you also another way, have a look at the solutions for the optimal depletion probelm for task 1, this time using [Pyomo](http://www.pyomo.org/). Here is a short [youtube-tutorial](https://www.youtube.com/watch?v=pxCogCylmKs) which uses Gurobi (a different solver engine). I suggest using ipopt as a solver in the exercise instead of gurobi. 
Especially in the beginning, Pyomo can appear more strucutred and following the definition of optimization components set at the very beginning (parameters, state variables, decision variables etc.). Before you start modelling, make sure you have
- run the cell below (enabling ipopt as a solver).
- installed pyomo as a package (maybe not necessary and package directly callable on GoogleColab)


In [ ]:
# ENABLE SOLVERS ON GOOGLE COLAB
# > to be able to run the optimization problems on Google Colab, make sure you run this cell first. This enables the use of the ipopt solver, later applied by the pyomo package.
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin 
    os.environ['PATH'] += ':bin'

UsageError: Line magic function `%%capture` not found.


**4b)**\
Rebuild 2b) using pyomo

In [21]:
import pyomo.environ as pyo
from pyomo.environ import *

# Initial parameters ----------------------------------------------------
r = 0.05
R0 = 150

# Create a model - the rest comes after
model = ConcreteModel()

# Define the SETs ----------------------------------------------------
# the subscripts in your Lagrangian, here indicating the time steps
model.t = RangeSet(0, 10)

# Define the PARAMETERS ----------------------------------------------------
# the parts of your optimization problem that change over time but in a CONSTANT manner
model.rho_t = Param(model.t, initialize=lambda model, t: (1/(1+r))**t, within=NonNegativeReals)

# Define the VARIABLES ----------------------------------------------------
# your decision variables which you can alter to optimze 
model.q = Var(model.t, domain=NonNegativeReals)
# the parts of your model which changes over time but in a NON-CONSTANT manner (different than in Excel)
model.R = Var(model.t, domain=NonNegativeReals)

# Define the CONSTRAINTS ----------------------------------------------------

# first, the "actual" constraints
def total_extraction_constraint(model):
    return sum(model.q[t] for t in model.t) <= R0
model.total_extraction_constraint = Constraint(rule=total_extraction_constraint)

# second, the constraints for Variables that are not actively altered by you, but change in a non-constant manner
def resource_constraint_rule(model, t):
    if t == 0:
        return model.R[t] == R0 
    else:
        return model.R[t] == model.R[t-1] - model.q[t-1]
model.resource_constraint = Constraint(model.t, rule=resource_constraint_rule)


# Define the OBJECTIVE FUNCTION ----------------------------------------------------
def obj_expression(model):
    return sum(model.rho_t[t]*log(1+model.q[t]) if t < 10 else 0  # this "else 0" statement is required as syntax by pyomo
               for t in model.t)
model.OBJ = Objective(rule=obj_expression, sense=maximize)

# SOLVE the model ----------------------------------------------------
solver = SolverFactory('ipopt')
results = solver.solve(model)

# report your extraction values from q such that you can compare them to the Excel model. still not nice to look at
# print(model.q.get_values())

# EXPORT more readable results ----------------------------------------------------
# use a data frame to report the results in a more readable manner
t_val = list(model.t)
q_val = [model.q[t].value for t in t_val]
R_val = [model.R[t].value for t in t_val]
profit_val = [log(1+model.q[t].value) for t in t_val]
NPV_profit_val = [model.rho_t[t]*log(1+model.q[t].value) for t in t_val]


df = pd.DataFrame({'t': t_val, 'q': q_val, 'R': R_val, 'profit': profit_val, 'NPV_profit': NPV_profit_val})

print(df)
print(f'\nsum: q={sum(df["q"]):.2f}, \t profit={sum(df["profit"]):.2f} \t NPV_profit={sum(df["NPV_profit"]):.2f}')


     t             q             R        profit    NPV_profit
0    0  1.873403e+01  1.500000e+02  2.982345e+00  2.982345e+00
1    1  1.779431e+01  1.312660e+02  2.933554e+00  2.793861e+00
2    2  1.689935e+01  1.134717e+02  2.884764e+00  2.616566e+00
3    3  1.604700e+01  9.657231e+01  2.835974e+00  2.449821e+00
4    4  1.523524e+01  8.052531e+01  2.787184e+00  2.293023e+00
5    5  1.446213e+01  6.529007e+01  2.738394e+00  2.145603e+00
6    6  1.372584e+01  5.082795e+01  2.689604e+00  2.007024e+00
7    7  1.302461e+01  3.710211e+01  2.640813e+00  1.876777e+00
8    8  1.235677e+01  2.407750e+01  2.592023e+00  1.754383e+00
9    9  1.172073e+01  1.172073e+01  2.543233e+00  1.639391e+00
10  10  7.546266e-07  4.315797e-08  7.546263e-07  4.632751e-07

sum: q=150.00, 	 profit=27.63 	 NPV_profit=22.56


**4c)**\
Rebuild 2c) using pyomo

In [22]:
import pyomo.environ as pyo
from pyomo.environ import *

# Initial parameters ----------------------------------------------------
r = 0.05
R0 = 100

# Create a model - the rest comes after
model = ConcreteModel()

# Define the SETs ----------------------------------------------------
# the subscripts in your Lagrangian, here indicating the time steps
model.t = RangeSet(0, 10)

# Define the PARAMETERS ----------------------------------------------------
# the parts of your optimization problem that change over time but in a CONSTANT manner
model.rho_t = Param(model.t, initialize=lambda model, t: (1/(1+r))**t, within=NonNegativeReals)

# Define the VARIABLES ----------------------------------------------------
# your decision variables which you can alter to optimze 
model.q = Var(model.t, domain=NonNegativeReals)
# the parts of your model which changes over time but in a NON-CONSTANT manner (different than in Excel)
model.R = Var(model.t, domain=NonNegativeReals)

# Define the CONSTRAINTS ----------------------------------------------------

# first, the "actual" constraints
def total_extraction_constraint(model):
    return sum(model.q[t] for t in model.t) <= R0
model.total_extraction_constraint = Constraint(rule=total_extraction_constraint)

# second, the constraints for Variables that are not actively altered by you, but change in a non-constant manner
def resource_constraint_rule(model, t):
    if t == 0:
        return model.R[t] == R0 
    else:
        return model.R[t] == model.R[t-1] - model.q[t-1]
model.resource_constraint = Constraint(model.t, rule=resource_constraint_rule)


# ADDITIONAL specific constraint for this problem <-----------------------------------
def max_q_capacity(model,t): 
    return model.q[t] <= 8
model.max_q_capacity = Constraint(model.t, rule=max_q_capacity)


# Define the OBJECTIVE FUNCTION ----------------------------------------------------
def obj_expression(model):
    return sum(model.rho_t[t]*log(1+model.q[t]) if t < 10 else 0  # this "else 0" statement is required as syntax by pyomo
               for t in model.t)
model.OBJ = Objective(rule=obj_expression, sense=maximize)

# SOLVE the model ----------------------------------------------------
solver = SolverFactory('ipopt')
results = solver.solve(model)

# report your extraction values from q such that you can compare them to the Excel model. still not nice to look at
# print(model.q.get_values())

# EXPORT more readable results ----------------------------------------------------
# use a data frame to report the results in a more readable manner
t_val = list(model.t)
q_val = [model.q[t].value for t in t_val]
R_val = [model.R[t].value for t in t_val]
profit_val = [log(1+model.q[t].value) for t in t_val]
NPV_profit_val = [model.rho_t[t]*log(1+model.q[t].value) for t in t_val]


df = pd.DataFrame({'t': t_val, 'q': q_val, 'R': R_val, 'profit': profit_val, 'NPV_profit': NPV_profit_val})

print(df)
print(f'\nsum: q={sum(df["q"]):.2f}, \t profit={sum(df["profit"]):.2f} \t NPV_profit={sum(df["NPV_profit"]):.2f}')


     t         q           R    profit  NPV_profit
0    0  8.000000  100.000000  2.197225    2.197225
1    1  8.000000   92.000000  2.197225    2.092595
2    2  8.000000   84.000000  2.197225    1.992947
3    3  8.000000   76.000000  2.197225    1.898045
4    4  8.000000   68.000000  2.197225    1.807662
5    5  8.000000   60.000000  2.197225    1.721583
6    6  8.000000   52.000000  2.197225    1.639603
7    7  8.000000   44.000000  2.197225    1.561526
8    8  8.000000   36.000000  2.197225    1.487168
9    9  8.000000   28.000000  2.197225    1.416351
10  10  1.298697   19.999999  0.832343    0.510986

sum: q=81.30, 	 profit=22.80 	 NPV_profit=18.33


**4d)**\
Rebuild 2d) using pyomo

In [23]:
import pyomo.environ as pyo
from pyomo.environ import *

# Initial parameters ----------------------------------------------------
r = 0.05
R0 = 100
RT = 27

# Create a model - the rest comes after
model = ConcreteModel()

# Define the SETs ----------------------------------------------------
# the subscripts in your Lagrangian, here indicating the time steps
model.t = RangeSet(0, 10)

# Define the PARAMETERS ----------------------------------------------------
# the parts of your optimization problem that change over time but in a CONSTANT manner
model.rho_t = Param(model.t, initialize=lambda model, t: (1/(1+r))**t, within=NonNegativeReals)

# Define the VARIABLES ----------------------------------------------------
# your decision variables which you can alter to optimze 
model.q = Var(model.t, domain=NonNegativeReals)
# the parts of your model which changes over time but in a NON-CONSTANT manner (different than in Excel)
model.R = Var(model.t, domain=NonNegativeReals)

# Define the CONSTRAINTS ----------------------------------------------------

# first, the "actual" constraints
def total_extraction_constraint(model):
    return sum(model.q[t] for t in model.t) <= R0 - RT
model.total_extraction_constraint = Constraint(rule=total_extraction_constraint)

# second, the constraints for Variables that are not actively altered by you, but change in a non-constant manner
def resource_constraint_rule(model, t):
    if t == 0:
        return model.R[t] == R0 
    else:
        return model.R[t] == model.R[t-1] - model.q[t-1]
model.resource_constraint = Constraint(model.t, rule=resource_constraint_rule)


# Define the OBJECTIVE FUNCTION ----------------------------------------------------
def obj_expression(model):
    return sum(model.rho_t[t]*log(1+model.q[t]) if t < 10 else 0  # this "else 0" statement is required as syntax by pyomo
               for t in model.t)
model.OBJ = Objective(rule=obj_expression, sense=maximize)

# SOLVE the model ----------------------------------------------------
solver = SolverFactory('ipopt')
results = solver.solve(model)

# report your extraction values from q such that you can compare them to the Excel model. still not nice to look at
# print(model.q.get_values())

# EXPORT more readable results ----------------------------------------------------
# use a data frame to report the results in a more readable manner
t_val = list(model.t)
q_val = [model.q[t].value for t in t_val]
R_val = [model.R[t].value for t in t_val]
profit_val = [log(1+model.q[t].value) for t in t_val]
NPV_profit_val = [model.rho_t[t]*log(1+model.q[t].value) for t in t_val]


df = pd.DataFrame({'t': t_val, 'q': q_val, 'R': R_val, 'profit': profit_val, 'NPV_profit': NPV_profit_val})

print(df)
print(f'\nsum: q={sum(df["q"]):.2f}, \t profit={sum(df["profit"]):.2f} \t NPV_profit={sum(df["NPV_profit"]):.2f}')


     t             q           R        profit    NPV_profit
0    0  9.237028e+00  100.000000  2.326011e+00  2.326011e+00
1    1  8.749551e+00   90.762972  2.277221e+00  2.168782e+00
2    2  8.285287e+00   82.013421  2.228431e+00  2.021253e+00
3    3  7.843130e+00   73.728134  2.179641e+00  1.882856e+00
4    4  7.422029e+00   65.885004  2.130851e+00  1.753056e+00
5    5  7.020980e+00   58.462976  2.082061e+00  1.631349e+00
6    6  6.639028e+00   51.441996  2.033270e+00  1.517258e+00
7    7  6.275265e+00   44.802968  1.984480e+00  1.410333e+00
8    8  5.928824e+00   38.527703  1.935690e+00  1.310151e+00
9    9  5.598880e+00   32.598879  1.886900e+00  1.216313e+00
10  10  1.565150e-08   26.999999  1.565150e-08  9.608664e-09

sum: q=73.00, 	 profit=21.06 	 NPV_profit=17.24
